In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Generator model
def build_generator(latent_dim, channels):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(channels), activation='tanh'))
    model.add(layers.Reshape((*channels,)))

    return model

# Discriminator model
def build_discriminator(image_shape):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=image_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Combine generator and discriminator into a GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# Hyperparameters
latent_dim = 100
image_shape = (64, 64, 3)

# Build and compile the models
generator = build_generator(latent_dim, image_shape[-1])
discriminator = build_discriminator(image_shape)
gan = build_gan(generator, discriminator)

discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0002, 0.5))


# Training loop
epochs = 10000
batch_size = 32

for epoch in range(epochs):
    # Select a random batch of hand drawings
    idx = np.random.randint(0, dataset.shape[0], batch_size)
    real_images = dataset[idx]

    # Generate a batch of digital drawings
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generated_images = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator (trying to make the discriminator classify generated images as real)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print progress
    print(f"Epoch {epoch}/{epochs} - D loss: {d_loss[0]} - D accuracy: {100 * d_loss[1]}% - G loss: {g_loss}")

    # Optionally, save generated images at certain intervals
    if epoch % save_interval == 0:
        save_generated_images(epoch)
